<a href="https://colab.research.google.com/github/ishankaul1/cs5262/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Background**

Following the recent pandemic, the tumultuous world economy, and major wars throughout the globe, mental health is more than ever a huge topic of discussion in modern society. CDC data shows that between 2019 and 2021, rates of mental health care have increased by 5% among adults in the US. In addition,tech companies such as Calm (valued over $2 billion) have cashed out on the new emphasis on mental health. As an individual who highly values my own mental health and takes great care to optimize and regulate my daily mood, I wanted to explore what features of someone's lifestyle.


**Description**

This project will explore a lifestyle and wellbeing [dataset](https://www.kaggle.com/datasets/ydalat/lifestyle-and-wellbeing-data) I found on Kaggle, to determine what factors about someone's lifestyle might affect their mental well-being. Although the definition of what a 'good' or 'bad' state of mental health might be is subjective, I plan on using a combination of the two dataset columns 'Daily Stress', and 'Daily Shouting' as my indicator of how good or bad an individual's mental health is.


**Performance Metric**

I will be formulating the Mental Health score as a binary classifier, with the value dependent upon both 'Daily Stress' and 'Daily Shouting'. Based on my initial analysis of the data for both columns, I will consider a '1' in the Mental Health score to be 

```
Daily_Shouting > 1 OR Daily_Stress > 2
```
This is subject to change, and I may end up using a more elaborate formula to determine the binary value as I research more ways to combine data.
The main performance metric I'll be using is F1 score on predicting the Mental Health score, as it seems to be a widely accepted assessment of model results. However, I will definitely take a look at precision, recall, accuracy, and other performance metrics we learn in class for further analysis on my models. 
